In [2]:
import pandas as pd
import requests
import time
from sklearn.linear_model import LinearRegression

# === CONFIGURAÇÕES DO THINGSPEAK ===
CHANNEL_ID = '3058667'
READ_API_KEY = 'A923CJZ5DMV9RQMX'
WRITE_API_KEY = 'U07M4R9UPNNHLDYX'
THINGSPEAK_URL = 'https://api.thingspeak.com/update'
CSV_URL = f'https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.csv?api_key={READ_API_KEY}&results=15000'

print("⏳ Aguardando 18 segundos para garantir que os dados estejam atualizados...")
time.sleep(18)

# === CARREGAR DADOS DO CSV ===
try:
    df = pd.read_csv(CSV_URL)
except Exception as e:
    print("❌ Erro ao carregar dados:", e)
    exit()

# === REMOVER COLUNAS DUPLICADAS ===
df = df.loc[:, ~df.columns.duplicated()]
print("📋 Colunas únicas disponíveis:", df.columns.tolist())

# === ESCOLHA DA UMIDADE DE REFERÊNCIA ===
print("\n💧 Escolha a fonte de umidade do solo para previsão:")
print("1 - Umidade geral (field1)")
print("2 - Umidade da alface (field8)")
opcao = input("Digite 1 ou 2: ").strip()

# === CAMPOS BASE ===
todos_campos = ['field1', 'field2', 'field3', 'field4', 'field5', 'field8']
campos_disponiveis = [c for c in todos_campos if c in df.columns]
df = df[['created_at'] + campos_disponiveis].copy()
campos = campos_disponiveis

# === DEFINIR CAMPO DE UMIDADE E PARÂMETROS DE IRRIGAÇÃO ===
if opcao == '2' and 'field8' in campos:
    campo_umidade = 'field8'
    TEMPO_MAXIMO = 30
    TEMPO_ESPERA = 300
    print("✅ Usando umidade da alface (field8). Parâmetros ajustados.")
else:
    campo_umidade = 'field1'
    TEMPO_MAXIMO = 60
    TEMPO_ESPERA = 120
    print("✅ Usando umidade geral (field1). Parâmetros padrão mantidos.")

# === TRATAMENTO DE DADOS ===
df.replace('NULL', 0, inplace=True)
for campo in campos:
    df[campo] = pd.to_numeric(df[campo], errors='coerce').fillna(0)
    df[campo] = df[campo].apply(lambda x: max(0, x))

if df.empty or len(df) < 100:
    print("❌ Dados insuficientes para treinar o modelo.")
    exit()

df['dias'] = df.index

# === CÁLCULO DE ECONOMIA ===
VAZAO_LPS = 0.05
df['economia'] = (TEMPO_MAXIMO + TEMPO_ESPERA - df['field5']) * VAZAO_LPS
df['economia'] = df['economia'].apply(lambda x: max(0, x))

df = df[df['economia'] > 0]
if df.empty:
    print("❌ Nenhum dado com economia positiva encontrado.")
    exit()

# === TREINAMENTO DO MODELO ===
X = df[['dias'] + campos]
y = df['economia']
modelo = LinearRegression()
modelo.fit(X, y)

# === PREVISÃO FUTURA ===
dias_atual = df['dias'].max()
economia_acumulada = 0

print("\n📊 Previsões acumulativas de economia de água (Litros):")
for ano in range(1, 5):
    dias_futuros = dias_atual + (365 * ano)
    entrada = df[campos].mean().to_dict()

    entrada[campo_umidade] *= (1 + 0.1 * ano)  # mais umidade
    entrada['field4'] *= (1 + 0.1 * ano)       # mais chuva
    entrada['field5'] *= (1 - 0.1 * ano)       # menos irrigação
    entrada['dias'] = dias_futuros

    entrada_df = pd.DataFrame([entrada])[X.columns]
    economia_ano = modelo.predict(entrada_df)[0]
    economia_ano = round(max(0, economia_ano), 2)

    economia_acumulada += economia_ano
    print(f"🗓️ Até o ano {ano}: economia acumulada = {economia_acumulada:.2f} L")

    # Enviar para field7
    url = f"{THINGSPEAK_URL}?api_key={WRITE_API_KEY}&field7={economia_acumulada}"
    try:
        r = requests.get(url)
        if r.status_code == 200 and r.text != '0':
            print(f"✅ Previsão enviada para field7: {economia_acumulada:.2f} L (Ano {ano})")
        else:
            print(f"❌ Falha ao enviar. Status: {r.status_code}, Resposta: {r.text}")
    except Exception as e:
        print(f"❌ Erro na requisição para Ano {ano}:", e)

    time.sleep(15)


⏳ Aguardando 18 segundos para garantir que os dados estejam atualizados...
📋 Colunas únicas disponíveis: ['created_at', 'entry_id', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7', 'field8']

💧 Escolha a fonte de umidade do solo para previsão:
1 - Umidade geral (field1)
2 - Umidade da alface (field8)
Digite 1 ou 2: 2
✅ Usando umidade da alface (field8). Parâmetros ajustados.

📊 Previsões acumulativas de economia de água (Litros):
🗓️ Até o ano 1: economia acumulada = 16.49 L
✅ Previsão enviada para field7: 16.49 L (Ano 1)
🗓️ Até o ano 2: economia acumulada = 32.98 L
✅ Previsão enviada para field7: 32.98 L (Ano 2)
🗓️ Até o ano 3: economia acumulada = 49.48 L
✅ Previsão enviada para field7: 49.48 L (Ano 3)
🗓️ Até o ano 4: economia acumulada = 65.98 L
✅ Previsão enviada para field7: 65.98 L (Ano 4)
